## Installation



In [ ]:
pip install wandb numpy pandas matplotlib torch torchvision

## Dataset

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#!unzip /content/drive/MyDrive/DL_Assignment2/Dataset/nature_12K.zip -d /content/drive/MyDrive/DL_Assignment2/Dataset

## Libraries

In [19]:
import torch
import os
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from torch.utils.data import Subset, DataLoader
import numpy as np
import torch.nn as nn
import torch.optim as optim
import wandb
from torchvision.models import efficientnet_v2_s, EfficientNet_V2_S_Weights
import gc
from PIL import Image


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

cuda


## Dataset loader

In [20]:
def validationDataSplit(train_dataset):
  classLabels = [label for _,label in train_dataset.samples]
  num_classes = len(np.unique(classLabels))

  sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
  train_indices, val_indices = next(sss.split(train_dataset.samples, classLabels))

  train_subset = Subset(train_dataset, train_indices)
  val_subset = Subset(train_dataset, val_indices)
  return train_subset, val_subset, num_classes


def load_data(base_dir, isDataAug, batch_size):
  train_dir = os.path.join(base_dir, 'train')
  test_dir = os.path.join(base_dir, 'val')

  train_transform, test_transform = None, None

  # model_transforms = EfficientNet_V2_S_Weights.IMAGENET1K_V1.transforms()
  # print(model_transforms)
    
  if isDataAug == False:
    train_transform = transforms.Compose([
      transforms.Resize(384, interpolation=Image.BILINEAR),
      transforms.CenterCrop(384),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
  else:
    train_transform = transforms.Compose([
      transforms.Resize(384, interpolation=Image.BILINEAR),
      transforms.CenterCrop(384),
      transforms.RandomHorizontalFlip(),
      transforms.RandomRotation(10),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

  test_transform = transforms.Compose([
      transforms.Resize(384, interpolation=Image.BILINEAR),
      transforms.CenterCrop(384),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

  train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
  test_dataset = datasets.ImageFolder(test_dir, transform=test_transform)
  train_dataset, val_dataset, num_classes = validationDataSplit(train_dataset)

  # print(f"inp: {train_dataset[0][0].shape} {train_dataset[0][1]}")

  train_loader = DataLoader(train_dataset,shuffle=True,num_workers=2,batch_size=batch_size,pin_memory=True)
  test_loader = DataLoader(test_dataset,shuffle=True,num_workers=2,batch_size=64,pin_memory=True)
  val_loader = DataLoader(val_dataset,shuffle=True,num_workers=2,batch_size=64,pin_memory=True)

  return train_loader, test_loader, val_loader, num_classes

# load_data("/content/drive/MyDrive/DL_Assignment2/Dataset/inaturalist_12K/", True)

## Network

In [9]:
class ConvolutionalNeuralNetwork(nn.Module):
  activationFunctionsMap = {"ReLU": nn.ReLU, "GELU": nn.GELU, "SiLU": nn.SiLU}
  # optimizersMap = {"sgd": optim.SGD, "rmsprop": optim.RMSprop, "adam": optim.Adam}

  def __init__(self, num_classes,
               num_filters, filter_sizes,
               activationFun, optimizer,
               n_neurons_denseLayer,
               isBatchNormalization, dropout,
               learning_rate=0.001,
               momentum=0.5, beta = 0.9,
               beta1=0.9, beta2=0.99,
               epsilon=1e-8, weight_decay=0.0001):
    super(ConvolutionalNeuralNetwork, self).__init__()
    self.num_classes = num_classes
    self.num_filters = num_filters
    self.filter_sizes = filter_sizes
    self.activationFun = ConvolutionalNeuralNetwork.activationFunctionsMap[activationFun]
    # self.optimizer = ConvolutionalNeuralNetwork.optimizersMap[optimizer]

    self.n_neurons_denseLayer = n_neurons_denseLayer
    self.isBatchNormalization = isBatchNormalization
    self.dropout = dropout

    self.lr = learning_rate
    self.momentum = momentum
    self.betas = (beta1, beta2)
    self.eps = epsilon
    self.alpha = beta
    self.weight_decay = weight_decay

    self.defineModel()

    trainable_parameters = [p for p in self.parameters() if p.requires_grad == True]

    if(optimizer == "sgd"):
      self.optimizer = optim.SGD(trainable_parameters, lr=self.lr, momentum=self.momentum, weight_decay=self.weight_decay)
    elif(optimizer == "rmsprop"):
      self.optimizer = optim.RMSprop(trainable_parameters, lr=self.lr, alpha=self.alpha, eps=self.eps, weight_decay=self.weight_decay)
    elif(optimizer == "adam"):
      self.optimizer = optim.Adam(trainable_parameters, lr=self.lr, betas=self.betas, eps=self.eps, weight_decay=self.weight_decay)


  def defineModel(self):
    self.model = efficientnet_v2_s(weights="IMAGENET1K_V1")

    # Freezing all layers except the last layer
    for param in self.model.parameters():
        param.requires_grad = False

    # Replacing last layer (its a classifier at the end containing dropout followed by linear layer)
    num_in_features_last_layer = self.model.classifier[1].in_features
    self.model.classifier[1] = nn.Linear(num_in_features_last_layer, self.num_classes)

    # last layer is trainable
    for name, param in self.model.classifier[1].named_parameters():
        param.requires_grad = True 
    
  def forward(self, inputs):
    return self.model(inputs)

  def backward(self, outputs, labels):
    loss = nn.CrossEntropyLoss()(outputs, labels)
    loss.backward()

  def updateWeights(self):
    self.optimizer.step()

## Training CNN

In [10]:
sweep_configuration = {
    "method": "random",
    "name" : "finetune_sweep1",
    "parameters": {
        "num_filters": {'values': [[32, 32, 32, 32, 32]]},  # have no effect since using pre-trained model
        "filter_sizes": {'values': [[3, 3, 3, 3, 3]]},        # have no effect since using pre-trained model
        "activation": {"values": ["ReLU"]},   # have no effect since using pre-trained model
        "optimizer": {"values": ["adam", "rmsprop", "sgd"]},
        "learning_rate": {"values": [1e-3, 1e-4]},
        "weight_decay": {"values": [0.0001, 0.0005]},
        "momentum": {"values": [0.9]},
        "beta": {"values": [0.9]},
        "beta1": {"values":[0.9]},
        "beta2": {"values": [0.999]},
        "epsilon": {"values": [1e-8]},
        # "base_dir": {"values":["/content/drive/MyDrive/DL_Assignment2/Dataset/inaturalist_12K/"]},
        "base_dir": {"values": ["/kaggle/input/inaturalist/inaturalist_12K"]},
        "isDataAug": {"values": ["False", "True"]},
        "isBatchNormalization": {"values": ["False"]},  # have no effect since using pre-trained model
        "dropout": {"values": [0]},            # have no effect since using pre-trained model
        "n_neurons_denseLayer": {"values": [128]},   # have no effect since using pre-trained model
        "batch_size": {"values": [32,64]},
        "epochs": {"values": [5,10]}
    }
}


def findOutputs(cnn, inputDataLoader):
  cnn.eval()  # setting the model to evaluation model
  outputs = []
  total_loss = 0.0
  n_correct = 0
  n_samples = 0

  with torch.no_grad():
    for batch_idx, (x_batch, y_batch) in enumerate(inputDataLoader):
      x_batch, y_batch = x_batch.to(device), y_batch.to(device)
      batch_outputs = cnn(x_batch)

      loss = nn.CrossEntropyLoss()(batch_outputs, y_batch)
      total_loss += loss.item() * x_batch.size(0)

      y_pred_batch = torch.argmax(batch_outputs, dim=1)
      n_correct += (y_pred_batch == y_batch).sum().item()
      n_samples += x_batch.size(0)

      outputs.append(batch_outputs)

  outputs = torch.cat(outputs)
  accuracy = (n_correct * 100.0) / n_samples
  avg_loss = total_loss / n_samples
  return outputs, accuracy, avg_loss

def trainNeuralNetwork_sweep():
  wandb.init(mode="online")
  args = wandb.config
  train_loader, test_loader, val_loader, num_classes = load_data(args["base_dir"], args["isDataAug"], args["batch_size"])
  activationFun = args["activation"]
  optimizer = args["optimizer"]
  learning_rate = args["learning_rate"]
  momentum = args["momentum"]
  beta = args["beta"]
  beta1 = args["beta1"]
  beta2 = args["beta2"]
  epsilon = args["epsilon"]
  weight_decay = args["weight_decay"]
  dropout = args["dropout"]
  num_filters = args["num_filters"]
  filter_sizes = args["filter_sizes"]
  n_neurons_denseLayer = args["n_neurons_denseLayer"]
  isBatchNormalization = args["isBatchNormalization"]
  isDataAug = args["isDataAug"]

  wandb.run.name = f"{activationFun}_{optimizer}_{dropout}_{n_neurons_denseLayer}_DataAug-{isDataAug}_BatchNorm-{isBatchNormalization}"
  best_val_accuracy = 0.0
  best_accuracy_epoch = -1

  cnn = ConvolutionalNeuralNetwork(num_classes,
                                   num_filters, filter_sizes,
                                   activationFun, optimizer,
                                   n_neurons_denseLayer,
                                   isBatchNormalization, dropout,
                                   learning_rate,
                                   momentum, beta,
                                   beta1, beta2,
                                   epsilon, weight_decay)
  cnn.to(device)

  epochs = args["epochs"]
  for epochNum in range(epochs):
    print(f"Epoch {epochNum}:")
    for batch_idx, (x_batch, y_batch) in enumerate(train_loader):
      if(batch_idx % 40 == 0):
        print(f"Batch idx {batch_idx} running")
      x_batch, y_batch = x_batch.to(device), y_batch.to(device)
      cnn.optimizer.zero_grad()
      outputs = cnn(x_batch)
      cnn.backward(outputs, y_batch)
      cnn.updateWeights()
      del x_batch, y_batch, outputs

    # Validation accuracy
    val_outputs, val_accuracy, val_loss = findOutputs(cnn, val_loader)
    # wandb.run.summary["metric_name"] = val_accuracy
    print(f"validation: loss={val_loss}, accuracy={val_accuracy}")

    # Train accuracy
    train_outputs, train_accuracy, train_loss = findOutputs(cnn, train_loader)
    print(f"training: loss={train_loss}, training={train_accuracy}")

    if val_accuracy > best_val_accuracy:
      best_val_accuracy = val_accuracy
      best_accuracy_epoch = epochNum

    wandb.log({
        "epoch": epochNum + 1,
        "val_loss": val_loss,
        "val_accuracy": val_accuracy,
        "train_loss": train_loss,
        "train_accuracy": train_accuracy
        },commit=True)
    del val_outputs, train_outputs
    gc.collect()
    torch.cuda.empty_cache()      
      
  wandb.log({
      "best_acc_epoch": best_accuracy_epoch,
      "best_val_accuracy": best_val_accuracy
  })
  del cnn,train_loader, test_loader, val_loader
  gc.collect()
  torch.cuda.empty_cache()

  wandb.finish()

In [ ]:
wandb.login(key="73b92c51e182b0f526498c09acb70e0d4b8d6c14")
wandb_id = wandb.sweep(sweep_configuration, project="DA6401_Assignment2")
wandb.agent(wandb_id, function=trainNeuralNetwork_sweep)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: cx4wjcu4
Sweep URL: https://wandb.ai/nikhithaa-iit-madras/DA6401_Assignment2/sweeps/cx4wjcu4


wandb: Agent Starting Run: 38lt1nq4 with config:
wandb: 	activation: ReLU
wandb: 	base_dir: /kaggle/input/inaturalist/inaturalist_12K
wandb: 	batch_size: 64
wandb: 	beta: 0.9
wandb: 	beta1: 0.9
wandb: 	beta2: 0.999
wandb: 	dropout: 0
wandb: 	epochs: 10
wandb: 	epsilon: 1e-08
wandb: 	filter_sizes: [3, 3, 3, 3, 3]
wandb: 	isBatchNormalization: False
wandb: 	isDataAug: False
wandb: 	learning_rate: 0.001
wandb: 	momentum: 0.9
wandb: 	n_neurons_denseLayer: 128
wandb: 	num_filters: [32, 32, 32, 32, 32]
wandb: 	optimizer: sgd
wandb: 	weight_decay: 0.0005


Epoch 0:
Batch idx 0 running
Batch idx 40 running
Batch idx 80 running
Batch idx 120 running
validation: loss=1.5355721521377563, accuracy=76.5
training: loss=1.5387026366479308, training=74.94686835854482
Epoch 1:
Batch idx 0 running
Batch idx 40 running
Batch idx 80 running
Batch idx 120 running
validation: loss=1.1396934213638306, accuracy=79.1
training: loss=1.145153384340422, training=78.28478559819978
Epoch 2:
Batch idx 0 running
Batch idx 40 running
Batch idx 80 running
Batch idx 120 running
validation: loss=0.9412926721572876, accuracy=81.1
training: loss=0.9571790002795931, training=79.55994499312413
Epoch 3:
Batch idx 0 running
Batch idx 40 running
Batch idx 80 running
Batch idx 120 running
validation: loss=0.8427000675201416, accuracy=81.65
training: loss=0.8541564155867851, training=79.5099387423428
Epoch 4:
Batch idx 0 running
Batch idx 40 running
Batch idx 80 running
Batch idx 120 running
validation: loss=0.7708968935012818, accuracy=82.9
training: loss=0.7845634194236499

best_acc_epoch,▁
best_val_accuracy,▁
epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▄▅▅▆█▇███
train_loss,█▅▃▃▂▂▁▁▁▁
val_accuracy,▁▃▅▅▇▆▇▇██
val_loss,█▅▃▃▂▂▁▁▁▁
best_acc_epoch,8
best_val_accuracy,84.55
epoch,10
train_accuracy,82.13527


...

wandb: Agent Starting Run: 66pbg785 with config:
wandb: 	activation: ReLU
wandb: 	base_dir: /kaggle/input/inaturalist/inaturalist_12K
wandb: 	batch_size: 32
wandb: 	beta: 0.9
wandb: 	beta1: 0.9
wandb: 	beta2: 0.999
wandb: 	dropout: 0
wandb: 	epochs: 10
wandb: 	epsilon: 1e-08
wandb: 	filter_sizes: [3, 3, 3, 3, 3]
wandb: 	isBatchNormalization: False
wandb: 	isDataAug: True
wandb: 	learning_rate: 0.0001
wandb: 	momentum: 0.9
wandb: 	n_neurons_denseLayer: 128
wandb: 	num_filters: [32, 32, 32, 32, 32]
wandb: 	optimizer: sgd
wandb: 	weight_decay: 0.0001


Epoch 0:
Batch idx 0 running
Batch idx 40 running
Batch idx 80 running
Batch idx 120 running
Batch idx 160 running
Batch idx 200 running
Batch idx 240 running
validation: loss=2.133926139831543, accuracy=39.05
training: loss=2.135267309851014, training=39.46743342917865
Epoch 1:
Batch idx 0 running
Batch idx 40 running
Batch idx 80 running
Batch idx 120 running
Batch idx 160 running
Batch idx 200 running
Batch idx 240 running
validation: loss=1.9417626647949218, accuracy=63.45
training: loss=1.94345563861009, training=62.07025878234779
Epoch 2:
Batch idx 0 running
Batch idx 40 running
Batch idx 80 running
Batch idx 120 running
Batch idx 160 running
Batch idx 200 running
Batch idx 240 running
validation: loss=1.7737868156433105, accuracy=71.7
training: loss=1.7741988795237893, training=70.83385423177897
Epoch 3:
Batch idx 0 running
Batch idx 40 running
Batch idx 80 running
Batch idx 120 running
Batch idx 160 running
Batch idx 200 running
Batch idx 240 running
validation: loss=1.63369969

best_acc_epoch,▁
best_val_accuracy,▁
epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▅▆▇▇█████
train_loss,█▇▅▅▄▃▂▂▁▁
val_accuracy,▁▅▇▇██████
val_loss,█▇▆▅▄▃▂▂▁▁
best_acc_epoch,9
best_val_accuracy,80.05
epoch,10
train_accuracy,79.60995


wandb: Agent Starting Run: 0bhd69kw with config:
wandb: 	activation: ReLU
wandb: 	base_dir: /kaggle/input/inaturalist/inaturalist_12K
wandb: 	batch_size: 64
wandb: 	beta: 0.9
wandb: 	beta1: 0.9
wandb: 	beta2: 0.999
wandb: 	dropout: 0
wandb: 	epochs: 10
wandb: 	epsilon: 1e-08
wandb: 	filter_sizes: [3, 3, 3, 3, 3]
wandb: 	isBatchNormalization: False
wandb: 	isDataAug: True
wandb: 	learning_rate: 0.0001
wandb: 	momentum: 0.9
wandb: 	n_neurons_denseLayer: 128
wandb: 	num_filters: [32, 32, 32, 32, 32]
wandb: 	optimizer: rmsprop
wandb: 	weight_decay: 0.0005


Epoch 0:
Batch idx 0 running
Batch idx 40 running
Batch idx 80 running
Batch idx 120 running
validation: loss=1.7817630662918091, accuracy=72.85
training: loss=1.7867826358841903, training=71.32141517689712
Epoch 1:
Batch idx 0 running
Batch idx 40 running
Batch idx 80 running
Batch idx 120 running
validation: loss=1.3847933006286621, accuracy=79.4
training: loss=1.3962474603893786, training=76.70958869858732
Epoch 2:
Batch idx 0 running
Batch idx 40 running
Batch idx 80 running
Batch idx 120 running
validation: loss=1.1213312883377076, accuracy=80.25
training: loss=1.1326183318674512, training=78.30978872359044
Epoch 3:
Batch idx 0 running
Batch idx 40 running
Batch idx 80 running
Batch idx 120 running
validation: loss=0.9457722082138061, accuracy=81.7
training: loss=0.9624430434318435, training=79.42242780347543
Epoch 4:
Batch idx 0 running
Batch idx 40 running
Batch idx 80 running
Batch idx 120 running
validation: loss=0.8300574011802674, accuracy=82.3
training: loss=0.8476519100158

In [12]:
# import os
# print(os.listdir('/kaggle/input/inaturalist/'))

['inaturalist_12K']


In [13]:
model_transforms = EfficientNet_V2_S_Weights.IMAGENET1K_V1.transforms()
print(model_transforms)

ImageClassification(
    crop_size=[384]
    resize_size=[384]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)
